In [3]:
from common_functions import get_secret, ret_metabase, google_sheets, dwh_query
import os
import sqlalchemy
import requests
import psycopg2
import json
from pathlib import Path
import pandas as pd
import numpy as np
import pytz
import io
from io import BytesIO

In [4]:
data = ret_metabase("Egypt", 63605)
data

/home/ec2-user/service_account_key.json


,RETAILER_ID,Workflow ID,Date,Description,AGENT_ID,COUPON_VALUE
0,23304,2,2025-10-05,2500- مبيعات مدفوعات: عميل طلبات مناسب انه يكو...,10726,NaN
1,23304,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,10726,NaN
2,23304,7,2025-10-05,تسجيل بينات العميل,10726,NaN
3,25339,6,2025-10-05,تفعيل الآجل، مصاريف إدارية 1.75% في الاسبوع ، ...,10726,NaN
4,25339,7,2025-10-05,تسجيل بينات العميل ‎||‎ حد أقصى للأجل ‎6000,10726,NaN
...,...,...,...,...,...,...
9616,114702,2,2025-10-05,2500- مبيعات مدفوعات: عميل طلبات مناسب انه يكو...,3060,NaN
9617,115395,1,2025-10-05,عميل غير نشط اكتر من اسبوعين - مطلوب اوردر من...,3060,NaN
9618,116020,2,2025-10-05,2500- مبيعات مدفوعات: عميل طلبات مناسب انه يكو...,3060,NaN
9619,116020,1,2025-10-05,عميل نشط - مطلوب اوردر من بضاعة مكسب بقيمة1326,3060,NaN


In [5]:
data = data[data["COUPON_VALUE"].notnull()]

data = data.sort_values(by="COUPON_VALUE", ascending=True)
data

,RETAILER_ID,Workflow ID,Date,Description,AGENT_ID,COUPON_VALUE
4758,361790,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,3141,20.0
4776,540544,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,3141,20.0
4814,21993,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,3154,20.0
4817,42457,5,2025-10-05,عميل غير نشط على سوق التجار اكتر من 3 شهور؛ كو...,3154,20.0
4828,58804,5,2025-10-05,عميل نشط على سوق التجار اخر 60 يوم؛ كوبون خصم ...,3154,20.0
...,...,...,...,...,...,...
92,164942,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,10730,100.0
91,164935,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,10730,100.0
85,163925,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,10730,100.0
79,163520,5,2025-10-05,عميل جديد لسوق التجار؛ اعمل أول اوردر من سوق ا...,10730,100.0


In [6]:
coupons  = data[["RETAILER_ID", "COUPON_VALUE"]]
coupons = data[["RETAILER_ID", "COUPON_VALUE"]].rename(
    columns={
        "RETAILER_ID": "retailer_id",
        "COUPON_VALUE": "amount"
    }
)

coupons["reason_id"] = 69
coupons["type_id"] = 4
coupons

,retailer_id,amount,reason_id,type_id
4758,361790,20.0,69,4
4776,540544,20.0,69,4
4814,21993,20.0,69,4
4817,42457,20.0,69,4
4828,58804,20.0,69,4
...,...,...,...,...
92,164942,100.0,69,4
91,164935,100.0,69,4
85,163925,100.0,69,4
79,163520,100.0,69,4


In [9]:
coupons.to_excel("coupons.xlsx", index=False)


In [ ]:
import requests
JT = get_jwt()

url = "https://api.maxab.info/commerce/api/admins/v1/wallet/transactions"

# Prepare file and form data
files = {
    "file": ("test.xlsx", open("coupons.xlsx", "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
}
data = {
    "compensation_coupon_rule_id": "343",
    "incentive_coupon_rule_id": "344"
}

headers = {
    "Authorization": f"Bearer {JT}"  # <-- replace with your actual token
}

# Perform the POST request
response = requests.post(url, headers=headers, files=files, data=data)

print(response.status_code)
print(response.text)


In [2]:
def get_jwt(country='EG'):
    import jwt
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]
    payload = {
            "client_id":"admin-portal",
            "grant_type":"password",    
            "username":fintech_service_account_emailname,
            "password":fintech_service_account_emailpass 
        }
 
    r = requests.post("https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token",
        headers={"Content-Type":"application/x-www-form-urlencoded"},
        data=payload
        )

    fullJwtResponse = r.json()
    jwt_access_token = fullJwtResponse['access_token']
        
    return jwt_access_token


In [6]:
    fintech_service_account = json.loads(get_secret("prod/fintechServiceEmail/credentials"))
    fintech_service_account_emailname= fintech_service_account["email_name"]
    fintech_service_account_emailpass= fintech_service_account["email_password"]

In [7]:
fintech_service_account_emailname


'fintech-lambda@maxab.io'

In [11]:
get_jwt()

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4cnE5b1ZqaXRFdjdNeExyTWpzS3NkdUl3eWdxZnZlVkM4T0Vfb3dvY3N3In0.eyJleHAiOjE3NTk0MTMyODEsImlhdCI6MTc1OTQxMjk4MSwianRpIjoiZWNjYWVlNTktOGE0Mi00Y2QzLWE4MGYtZmEzN2RkNjJiMmYwIiwiaXNzIjoiaHR0cHM6Ly9zc28ubWF4YWIuaW5mby9hdXRoL3JlYWxtcy9tYXhhYiIsImF1ZCI6WyJ3b3JrZm9yY2UiLCJlLW1vbmV5Iiwib3JkZXItbWFuYWdlbWVudCIsImNyZWRpdCIsIm1heGFiLWNvbW1lcmNlIiwiYWNjb3VudCJdLCJzdWIiOiJkYzVkZjcwMS1mODdkLTQwODUtODdmYy01YzI0ZWYzYTRiOGQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJhZG1pbi1wb3J0YWwiLCJzZXNzaW9uX3N0YXRlIjoiNjgxZjA5M2MtZjQ4OS00MmRlLWJkYjEtYmVkOGQ2NWI0YjNhIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHA6Ly9sb2NhbGhvc3Q6NDIwMCIsImh0dHBzOi8vYWRtaW4ubWF4YWIuaW5mbyJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1tYXhhYiIsIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iXX0sInJlc291cmNlX2FjY2VzcyI6eyJ3b3JrZm9yY2UiOnsicm9sZXMiOlsib3JkZXItbWFuYWdtZW50LWFwaXMtaW50ZWdyYXRpb24iLCJ3b3JrZm9yY2UtaW50ZWdyYXRpb24iXX0sImUtbW9uZXkiOnsicm9sZXMiOlsiZS1tb25leSJdfSwib3JkZXItbWFuYWdlbWVudCI6eyJyb2x